# Partitioning the input files

ROMS requires partitioned (or tiled) input files so that the simulation can be parallelized over multiple nodes. `ROMS-Tools` can create these partitioned files for you.

## Partitioning existing files

Let's assume we have already saved a global (i.e., non-partitioned) input file to disk. The following function can partition that global file.

In [1]:
from roms_tools.utils import partition_netcdf

Here is an example for creating and saving a global grid file with `ROMS-Tools`. (Note, however, that `ROMS-Tool`'s `partition_netcdf` tool does not care whether or whether not `ROMS-Tools` was used to create the global file.)

In [2]:
from roms_tools import Grid

In [3]:
grid = Grid(
    nx=100, ny=100, size_x=1800, size_y=2400, center_lon=-21, center_lat=61, rot=20
)

In [4]:
filepath = "/glade/derecho/scratch/noraloose/examples/my_grid.nc"

In [5]:
grid.save(filepath)

Saving the following files:
/glade/derecho/scratch/noraloose/examples/my_grid.nc


We can now partition the saved file. We need to tell the `partition_netcdf` function what domain decomposition to use via the following two parameters:

* `np_eta` : The number of partitions along the `eta` direction.
* `np_xi` : The number of partitions along the `xi` direction.

In [6]:
partition_netcdf(filepath, np_eta=2, np_xi=5)

Saving the following files:
/glade/derecho/scratch/noraloose/examples/my_grid.0.nc
/glade/derecho/scratch/noraloose/examples/my_grid.1.nc
/glade/derecho/scratch/noraloose/examples/my_grid.2.nc
/glade/derecho/scratch/noraloose/examples/my_grid.3.nc
/glade/derecho/scratch/noraloose/examples/my_grid.4.nc
/glade/derecho/scratch/noraloose/examples/my_grid.5.nc
/glade/derecho/scratch/noraloose/examples/my_grid.6.nc
/glade/derecho/scratch/noraloose/examples/my_grid.7.nc
/glade/derecho/scratch/noraloose/examples/my_grid.8.nc
/glade/derecho/scratch/noraloose/examples/my_grid.9.nc


## Creating partitioned files directly

Instead of first saving a global file and then partitioning the written NetCDF file, we can also tell `ROMS-Tools` to write partitioned files directly. This will skip writing the global file.

In [7]:
grid = Grid(
    nx=100, ny=100, size_x=1800, size_y=2400, center_lon=-21, center_lat=61, rot=20
)

In [8]:
filepath = "/glade/derecho/scratch/noraloose/examples/my_grid.nc"

In [9]:
grid.save(filepath, np_eta=2, np_xi=5)

Saving the following files:
/glade/derecho/scratch/noraloose/examples/my_grid.0.nc
/glade/derecho/scratch/noraloose/examples/my_grid.1.nc
/glade/derecho/scratch/noraloose/examples/my_grid.2.nc
/glade/derecho/scratch/noraloose/examples/my_grid.3.nc
/glade/derecho/scratch/noraloose/examples/my_grid.4.nc
/glade/derecho/scratch/noraloose/examples/my_grid.5.nc
/glade/derecho/scratch/noraloose/examples/my_grid.6.nc
/glade/derecho/scratch/noraloose/examples/my_grid.7.nc
/glade/derecho/scratch/noraloose/examples/my_grid.8.nc
/glade/derecho/scratch/noraloose/examples/my_grid.9.nc
